# Autoatenção e codificação posicional
self-attention and positional encoding

### Objetivos
- Entenda os conceitos básicos de tokenização e como os dados textuais são preparados para modelos de rede neural.
- Aprenda o conceito de codificação one-hot e sua aplicação na representação de dados textuais para modelos de aprendizado de máquina.
- Explore o mecanismo de autoatenção, um componente-chave dos modelos Transformer, que permite que o modelo se concentre dinamicamente em diferentes partes da sequência de entrada para fazer previsões.
- Implemente um mecanismo básico de autoatenção e integre-o a um modelo de rede neural.
- Entenda a importância da codificação posicional em modelos Transformer, fornecendo ao modelo as informações necessárias sobre a ordem das palavras em uma frase.
- Implemente a codificação posicional e observe seu efeito no desempenho do modelo e na compreensão da ordem da sequência.
- Aplique os conceitos aprendidos para construir um modelo de tradução simples ou tarefa de processamento de texto, demonstrando a aplicação prática da autoatenção e da codificação posicional.
- Desenvolver uma intuição para o funcionamento dos modelos NLP modernos, particularmente a arquitetura Transformer, e entender suas vantagens sobre os modelos tradicionais de processamento de sequência, como RNNs e LSTMs.
- Incentivar maior exploração e experimentação com diferentes arquiteturas de modelos, hiperparâmetros e aplicações em processamento de linguagem natural.

### Self-attention (autoatenção)
Self-attention é um mecanismo usado em redes neurais para permitir que o modelo foque em diferentes partes dos dados de entrada ao gerar cada parte da saída. É um componente essencial da arquitetura Transformer, amplamente utilizado em tarefas de processamento de linguagem natural (NLP), como tradução automática, sumarização de texto e análise de sentimento.

A ideia por trás do self-attention é permitir que o modelo atribua pesos às partes importantes dos tokens de entrada ao gerar cada token de saída. Isso é feito computando uma soma ponderada dos tokens de entrada, onde os pesos são determinados pelas relações entre todos os pares de tokens de entrada.

#### Benefícios
- Contextualização: O modelo pode levar em consideração a relação entre palavras, mesmo que estejam distantes na sequência.
- Paralelismo: Ao contrário de RNNs, que processam sequências de forma sequencial, o self-attention pode ser calculado em paralelo, tornando o treinamento mais rápido e eficiente.
- Versatilidade: Pode ser usado em diversas tarefas, desde tradução de texto até tarefas de classificação.

Esse mecanismo forma a base do sucesso dos Transformers, como nos modelos BERT e GPT, que se destacam por sua habilidade em capturar relacionamentos contextuais em sequências de texto.

### Setup
Para este laboratório, vamos usar as seguintes bibliotecas:

- [`torch`](https://pytorch.org/): A biblioteca principal para construir e treinar modelos de rede neural neste projeto, incluindo a implementação de mecanismos de autoatenção e codificações posicionais.
- [`torch.nn`](https://pytorch.org/docs/stable/nn.html), [`torch.nn.functional`](https://pytorch.org/docs/stable/nn.functional.html): Esses submódulos do PyTorch são usados ​​para definir as camadas da rede neural e aplicar funções como ativações, que são essenciais na construção da arquitetura do modelo.
- [`Levenshtein`](https://pypi.org/project/python-Levenshtein/): Esta biblioteca é usada para calcular a distância de Levenshtein, que pode ser útil para avaliar o desempenho do modelo em tarefas como geração ou tradução de texto, medindo a diferença entre as sequências de texto previstas e reais.
- [`get_tokenizer`](https://pytorch.org/text/stable/data_utils.html), [`build_vocab_from_iterator`](https://pytorch.org/text/stable/vocab.html) de `torchtext`: Essas funções são cruciais para o pré-processamento de dados de texto, incluindo a tokenização de texto em palavras ou subpalavras e a construção de um vocabulário a partir do conjunto de dados, que são etapas fundamentais na preparação de dados para modelos de PNL.

In [1]:
def warn(*args, **kwargs):
    pass
    
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import importlib.util
import subprocess
import sys

def check_and_install(package, pip_name=None):
    if pip_name is None:
        pip_name = package
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} não está instalado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
    else:
        print(f"{package} já está instalado.")

In [2]:
check_and_install('Levenshtein')
# check_and_install('torch','torch==2.3.0')
# check_and_install('torchtext','torchtext==0.18.0')

Levenshtein não está instalado. Instalando...


### Importar bibliotecas

In [3]:
import os
import sys
import time
from pathlib import Path
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests

from Levenshtein import distance
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

- **Configuração do dispositivo**: Atribuímos os cálculos a uma GPU, se disponível, caso contrário, usamos a CPU. Utilizar uma GPU pode acelerar significativamente o treinamento de modelos de aprendizado profundo. Se CUDA estiver disponível, definiremos como `cuda`, caso contrário, definiremos como `cpu`. Compute Unified Device Architecture (CUDA) é uma plataforma de computação paralela e interface de programação de aplicativos (API) que permite que o software aproveite unidades de processamento gráfico (GPUs) específicas para processamento acelerado de propósito geral, conhecido como computação de propósito geral em GPUs.

- **Parâmetros de treinamento**:
- `learning_rate`: Este é o tamanho do passo em cada iteração enquanto se move em direção a um mínimo da função de perda. Definimos como `3e-4`, que é um ponto de partida comum para muitos modelos.
- `batch_size`: O número de amostras que serão propagadas pela rede em uma passagem para frente/para trás. Aqui, é `64`.
- `max_iters`: O número total de iterações de treinamento que planejamos executar. Defina como `5000` para permitir que o modelo tenha ampla oportunidade de aprender com os dados.
- `eval_interval` e `eval_iters`: Parâmetros que definem a frequência com que avaliamos o desempenho do modelo em um número definido de lotes para aproximar a perda.

- **Parâmetros de arquitetura**:
- `max_vocab_size`: Isso representa o número máximo de tokens em nosso vocabulário. Ele é definido como `256`, o que significa que consideraremos apenas os 256 tokens mais frequentes.
- `vocab_size`: O número real de tokens no vocabulário, que pode ser menor que o máximo devido ao comprimento variável de tokens na tokenização de subpalavras como BPE (Byte Pair Encoding).
- `block_size`: O comprimento da sequência de entrada que o modelo foi projetado para manipular. Aqui é `16`.
- `n_embd`: O tamanho de cada vetor de incorporação, definido como `32`. As incorporações convertem tokens em um espaço contínuo onde tokens semelhantes estão mais próximos uns dos outros.
- `num_heads`: O número de cabeças no mecanismo de autoatenção de várias cabeças, `2` neste caso, que permite que o modelo atenda conjuntamente às informações de diferentes subespaços de representação.
- `n_layer`: O número de camadas (ou profundidade) da rede. Aqui, `2` camadas são usadas.
- `ff_scale_factor`: Um fator de escala para o tamanho das redes feed-forward, escolhido como `4` aqui.
- `dropout`: A taxa de dropout usada para regularização para evitar overfitting, definida como `0.0`, indicando que não há dropout neste caso.

Finalmente, você tem um cálculo `head_size` que é derivado do tamanho de incorporação e do número de cabeças, garantindo que cada cabeça tenha um pedaço igual do tamanho de incorporação para trabalhar. Também incluímos uma asserção para verificar se `head_size` vezes `num_heads` é igual a `n_embd`.

In [4]:
# Device for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
split = 'train'

# Training parameters
learning_rate = 3e-4
batch_size = 64
max_iters = 5000              # Maximum training iterations
eval_interval = 200           # Evaluate model every 'eval_interval' iterations in the training loop
eval_iters = 100              # When evaluating, approximate loss using 'eval_iters' batches

# Architecture parameters
max_vocab_size = 256          # Maximum vocabulary size
vocab_size = max_vocab_size   # Real vocabulary size (e.g. BPE has a variable length, so it can be less than 'max_vocab_size')
block_size = 16               # Context length for predictions
n_embd = 32                   # Embedding size
num_heads = 2                 # Number of head in multi-headed attention
n_layer = 2                   # Number of Blocks
ff_scale_factor = 4           # Note: The '4' magic number is from the paper: In equation 2 uses d_model=512, but d_ff=2048
dropout = 0.0                 # Normalization using dropout# 10.788929 M parameters

head_size = n_embd // num_heads
assert (num_heads * head_size) == n_embd

### Funções auxiliares

In [5]:
# Function designed to visualize the learned embeddings in a #D space
# This function helps in understanding how the embeddings cluester and separate
# diferrent tokens, providing insight into what the model has learned

def plot_embdings(my_embdings,name,vocab):

  fig = plt.figure()
  ax = fig.add_subplot(111, projection='3d')

  # Plot the data points
  ax.scatter(my_embdings[:,0], my_embdings[:,1], my_embdings[:,2])

  # Label the points
  for j, label in enumerate(name):
      i=vocab.get_stoi()[label]
      ax.text(my_embdings[j,0], my_embdings[j,1], my_embdings[j,2], label)

  # Set axis labels
  ax.set_xlabel('X Label')
  ax.set_ylabel('Y Label')
  ax.set_zlabel('Z Label')

  # Show the plot
  plt.show()

### Programa para tradução literal

Na próxima parte, vamos explorar os conceitos fundamentais de tokenização e tradução por meio de um programa simples para tradução literal do francês para o inglês:

- Um `dicionário` é definido, mapeando palavras francesas para seus equivalentes em inglês, formando a base da nossa lógica de tradução.

In [6]:
dictionary = {
    'le': 'the'
    , 'chat': 'cat'
    , 'est': 'is'
    , 'sous': 'under'
    , 'la': 'the'
    , 'table': 'table'
}

- A função `tokenize` é responsável por dividir uma frase em palavras individuais.
- A função `translate` usa esta função `tokenize` para dividir a frase de entrada e então traduz cada palavra de acordo com o dicionário. As palavras traduzidas são concatenadas para formar a frase de saída.

In [7]:
# Function to split a sentence into tokens (words)
def tokenize(text):
    """
    This function takes a string of text as input and returns a list of words (tokens).
    It uses the split method, which by default splits on any whitespace, to tokenize the text.
    """
    return text.split()  # Split the input text on whitespace and return the list of tokens

In [8]:
# Function to translate a sentence from source to target language word by word
def translate(sentence):
    """
    This function translates a sentence by looking up each word's translation in a predefined dictionary.
    It assumes that every word in the sentence is a key in the dictionary.
    """
    out = ''  # Initialize the output string
    for token in tokenize(sentence):  # Tokenize the sentence into words
        # Append the translated word to the output string
        # This line assumes the dictionary contains a translation for every word in the input
        out += dictionary[token] + ' '
    return out.strip()  # Return the translated sentence, stripping any extra whitespace

In [9]:
translate("le chat est sous la table")

'the cat is under the table'

Este exemplo simples ilustra uma substituição palavra por palavra que, embora não seja sofisticada, fornece uma introdução aos métodos de tradução computacional.

### Melhoria: E se a 'chave' não estiver no dicionário?

O código apresenta uma melhoria para o programa de tradução, abordando o cenário quando uma palavra não existe em nosso dicionário:

- **Função find_closest_key**: Esta nova função tem como objetivo encontrar a chave mais próxima no dicionário para uma determinada palavra de consulta. Ela usa a **distância de Levenshtein** (uma medida da diferença entre duas sequências) para encontrar a chave do dicionário com a distância mínima para a consulta, sugerindo uma palavra semelhante se uma correspondência exata não for encontrada.
- **Função de tradução aprimorada**: A função `translate` foi atualizada para usar `find_closest_key`. Agora, em vez de traduzir tokens diretamente com base no dicionário, ela primeiro encontra a chave mais próxima para cada palavra tokenizada. Isso permite uma tradução mais robusta, especialmente ao encontrar palavras com pequenos erros de ortografia ou variações não presentes no dicionário.
- **Demonstração**: A função de tradução melhorada é demonstrada com a entrada "tables". Embora "tables" não esteja no dicionário, espera-se que a função encontre e use a chave mais próxima "table" para a tradução, gerando "table" em inglês.

Esta melhoria demonstra uma forma simples de tratamento de erros e correspondência fuzzy em sistemas de tradução, permitindo traduções mais flexíveis e tolerantes a falhas.

In [10]:
# Function to find the closest key in the dictionary to the given query word
def find_closest_key(query):
    """
    The function computes the Levenshtein distance between the query and each key in the dictionary.
    The Levenshtein distance is a measure of the number of single-character edits required to change one word into the other.
    """
    closest_key, min_dist = None, float('inf')  # Initialize the closest key and minimum distance to infinity
    for key in dictionary.keys():
        dist = distance(query, key)  # Calculate the Levenshtein distance to the current key
        if dist < min_dist:  # If the current distance is less than the previously found minimum
            min_dist, closest_key = dist, key  # Update the minimum distance and the closest key
    return closest_key  # Return the closest key found

In [11]:
# Function to translate a sentence from source to target language using the dictionary
def translate(sentence):
    """
    This function tokenizes the input sentence into words and finds the closest translation for each word.
    It constructs the translated sentence by appending the translated words together.
    """
    out = ''  # Initialize the output string
    for query in tokenize(sentence):  # Tokenize the sentence into words
        key = find_closest_key(query)  # Find the closest key in the dictionary for each word
        out += dictionary[key] + ' '  # Append the translation of the closest key to the output string
    return out.strip()  # Return the translated sentence, stripping any extra whitespace

In [12]:
translate("tables")

'table'

### Converter para rede neural

Transicionando da tradução básica para redes neurais, vamos começar definindo nossos vocabulários de entrada e saída e então passar para a codificação de nossos tokens:

- **Definição de vocabulário**: Dois vocabulários são criados a partir do dicionário — `vocabulary_in` para o idioma de origem (francês) e `vocabulary_out` para o idioma de destino (inglês). Esses vocabulários são as listas de palavras únicas obtidas das chaves e valores do dicionário, respectivamente, e são classificadas para manter uma ordem consistente.
- **Codificação one-hot**: A função `encode_one_hot` é introduzida para converter cada palavra no vocabulário em um vetor codificado one-hot. A codificação one-hot é um processo em que representa cada palavra como um vetor binário com um '1' na posição correspondente ao índice da palavra no vocabulário e '0's em outros lugares. Isso cria um vetor único de tamanho fixo para cada palavra, o que é essencial para o processamento de rede neural.
- **Demonstração de codificação**: Demonstre o processo de codificação one-hot aplicando `encode_one_hot` ao nosso vocabulário de entrada (`vocabulary_in`) e mostrando os vetores codificados para cada palavra. O mesmo processo é então aplicado ao vocabulário de saída (`vocabulary_out`).

Esta etapa é crítica no aprendizado de máquina, pois prepara nossos dados textuais para entrada em uma rede neural, permitindo que ela aprenda e faça previsões sobre nossos dados.

### Definir 'vocabularies'

In [14]:
# Create and sort the input vocabulary from the dictionary's keys
vocabulary_in = sorted(list(set(dictionary.keys())))
# Display the size and the sorted vocabulary for the input language
print(f"Vocabulary input ({len(vocabulary_in)}): {vocabulary_in}")

# Create and sort the output vocabulary from the dictionary's values
vocabulary_out = sorted(list(set(dictionary.values())))
# Display the size and the sorted vocabulary for the output language
print(f"Vocabulary output ({len(vocabulary_out)}): {vocabulary_out}")

Vocabulary input (6): ['chat', 'est', 'la', 'le', 'sous', 'table']
Vocabulary output (5): ['cat', 'is', 'table', 'the', 'under']


### Tokens codificados utilizando o codificador 'one hot'

In [15]:
# Function to convert a list of vocabulary words into one-hot encoded vectors
def encode_one_hot(vocabulary):
    vocabulary_size = len(vocabulary) # Get the size of the vocabulary
    one_hot = dict() # Initialize a dictionary to hold our one-hot encodings
    LEN = len(vocabulary) # the length of each one-hot encoded will be equal to the vocabulary

    # Iterate over the vocabulary to create a one-hot encoded vector for each word
    for i, key in enumerate(vocabulary):
        one_hot_vector = torch.zeros(LEN) # Start with a vector of zeros
        one_hot_vector[i] = 1 # Set the i-th position on 1 for the current word
        one_hot[key] = one_hot_vector # Map the word to its one-hot encoded vector
        print(f"{key}\t: {one_hot[key]}")
    
    # Return the dict of words and thei one-hot encoded vectors
    return one_hot

In [16]:
# Apply the one-hot encoding function to the input vocabulary and store the result
one_hot_in = encode_one_hot(vocabulary_in)

chat	: tensor([1., 0., 0., 0., 0., 0.])
est	: tensor([0., 1., 0., 0., 0., 0.])
la	: tensor([0., 0., 1., 0., 0., 0.])
le	: tensor([0., 0., 0., 1., 0., 0.])
sous	: tensor([0., 0., 0., 0., 1., 0.])
table	: tensor([0., 0., 0., 0., 0., 1.])


In [17]:
# Iterate over the one-hot encoded input vocabulary and print each vector
# This visualizes the one-hot representation for each word in the input vocabulary
for k, v in one_hot_in.items():
    print(f"E_{{ {k} }} = ", v)

E_{ chat } =  tensor([1., 0., 0., 0., 0., 0.])
E_{ est } =  tensor([0., 1., 0., 0., 0., 0.])
E_{ la } =  tensor([0., 0., 1., 0., 0., 0.])
E_{ le } =  tensor([0., 0., 0., 1., 0., 0.])
E_{ sous } =  tensor([0., 0., 0., 0., 1., 0.])
E_{ table } =  tensor([0., 0., 0., 0., 0., 1.])


In [18]:
# Apply the one-hot encoding function to the output vocabulary and store the result
# This time we're encoding the target language vocabulary
one_hot_out = encode_one_hot(vocabulary_out)

cat	: tensor([1., 0., 0., 0., 0.])
is	: tensor([0., 1., 0., 0., 0.])
table	: tensor([0., 0., 1., 0., 0.])
the	: tensor([0., 0., 0., 1., 0.])
under	: tensor([0., 0., 0., 0., 1.])


### Vamos criar um 'dicionário' usando multiplicação de matrizes

Agora estamos ilustrando como criar uma representação do nosso dicionário adequada para operações de rede neural:

- **Criação de matriz**: Usando `torch.stack` do PyTorch, converta os vetores codificados one-hot para vocabulários de entrada (`K`) e saída (`V`) em tensores. `K` é construído a partir dos vetores one-hot do vocabulário de entrada, e `V` a partir dos vetores do vocabulário de saída. Esses tensores podem ser pensados ​​como uma tabela de consulta que nosso modelo usará para associar tokens de entrada com tokens de saída.
- **Dicionário como matrizes**: Esta etapa traduz efetivamente nosso mapeamento de dicionário palavra a palavra em um formato amigável à rede neural. Cada linha em `K` corresponde a uma palavra no idioma de entrada representada como um vetor one-hot, e cada linha em `V` corresponde à respectiva palavra traduzida no idioma de saída.
- **Exemplo de consulta**: Um exemplo mostra como usar operações de matriz para encontrar uma tradução. Procure o vetor one-hot para a palavra "sous" do vocabulário de entrada (`q`). Em seguida, demonstre como encontrar sua tradução correspondente realizando a multiplicação de matriz com a transposição de `K` (ou seja, `q @ K.T`) para identificar o índice e, em seguida, use esse índice para selecionar a linha relevante de `V`. Este processo imita a pesquisa que você executaria em uma rede neural real durante tarefas de tradução.

Esta representação de matriz é um precursor para entender como arquiteturas de rede neural mais complexas, como aquelas que usam autoatenção, gerenciam traduções de token.

In [19]:
# Stacking the one-hot encoded vectors for input vocabulary to form a tensor
K = torch.stack([one_hot_in[k] for k in dictionary.keys()])

# K now represents a matrix of one-hot vectors for the input vocabulary
print(K)

tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]])


In [20]:
# Similarly, stack the one-hot encoded vectors for output vocabulary to form a tensor
V = torch.stack([one_hot_out[k] for k in dictionary.values()])

# V represents the corresponding matrix of one-hot vectors for the output vocabulary
print(V)

tensor([[0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])


In [23]:
# Demostrating how to look up a translation for a given word using matrix operations
# Here, we take the one-hot representation of 'sous' from the input vocabulary

q = one_hot_in['sous']
print('Query token: ', q)

Query token:  tensor([0., 0., 0., 0., 1., 0.])


In [24]:
# Use the index found from the key selection to fing the corresponding 
#  value vector in V (output dictionary matrix)
# This operation selects the row from V that is the translation of 'sous' in the output vocab

print("Select value (V):", q @ K.T @ V)

Select value (V): tensor([0., 0., 0., 0., 1.])


Query vector, K matrix, and V matrix:

$$
q = \left[\begin{matrix}
  0 & 0 & 0 & 0 & 1 & 0\\\\\\\\\\\\
\end{matrix}\right]
; \
K = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0\\\\
  1 & 0 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 0 & 1\\\\
\end{matrix}\right]
; \
V = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
$$


The operation $q \cdot K^T \cdot V$ allows us to build a dictionary-like structure from a set of vectors

This is an example on how to select the value from a query:

$$
q \cdot K^T \cdot V =
\left[\begin{matrix}
  0 & 0 & 0 & 0 & 1 & 0\\\\\\\\\\\\
\end{matrix}\right]
\cdot
\left[\begin{matrix}
  0 & 1 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 1 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 0 & 1\\\\
\end{matrix}\right]
\cdot
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
$$


$$
q \cdot K^T \cdot V =
%\hspace{2cm}
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0\\\\\\\\\\\\
\end{matrix}\right]
%\hspace{2.5cm}
\cdot
\left[\begin{matrix}
  0 & 0 & 0 & 1 & 0\\\\
  1 & 0 & 0 & 0 & 0\\\\
  0 & 1 & 0 & 0 & 0\\\\
  0 & 0 & 0 & 0 & 1\\\\
  0 & 0 & 0 & 1 & 0\\\\
  0 & 0 & 1 & 0 & 0\\\\
\end{matrix}\right]
\hspace{4.5cm}
$$


$$
q \cdot K^T \cdot V
=
%\hspace{3.5cm}
\left[\begin{matrix}
0 & 0 & 0 & 0 & 1\\\\\\\\\\\\
\end{matrix}\right]
%\hspace{3.5cm}
\hspace{9cm}
$$


O código a seguri introduz uma função para decodificar vetores one-hot para tokens e atualiza a função de tradução para utilizar multiplicação de matrizes:

### Decodificar vetor one-hot (decode one-hot vector)
A função `decode_one_hot` é projetada para decodificar um vetor codificado one-hot de volta para o token correspondente (palavra). Ela faz isso encontrando o token cuja representação one-hot tem a maior similaridade de cosseno com o vetor fornecido, que é efetivamente apenas o produto escalar devido à natureza dos vetores one-hot.

In [25]:
def decode_one_hot(one_hot, vector):
    """ 
    Decode a one-hot encoded vector to find the best matching token in the vocabulary.
    """
    best_key, best_cosine_sim = None, 0
    for k, v in one_hot.items():  # Iterate over the one-hot encoded vocabulary
        cosine_sim = torch.dot(vector, v)  # Calculate dot product (cosine similarity)
        if cosine_sim > best_cosine_sim:  # If this is the best similarity we've found
            best_cosine_sim, best_key = cosine_sim, k  # Update the best similarity and token
    return best_key  # Return the token corresponding to the one-hot vector

### Função de tradução baseada em matriz
A função `translate` agora alavanca operações de matriz para executar a tradução. Para cada token na frase de entrada, ela encontra seu vetor one-hot, multiplica-o pelas matrizes `K.T` e `V` para encontrar o vetor one-hot correspondente no vocabulário de saída e, em seguida, decodifica esse vetor para obter a palavra traduzida.

In [26]:
def translate(sentence):
    """ 
    Translate a sentence using matrix multiplication, treating the dictionaries as matrices.
    """
    sentence_out = ''  # Initialize the output sentence
    for token_in in tokenize(sentence):  # Tokenize the input sentence
        q = one_hot_in[token_in]  # Find the one-hot vector for the token
        out = q @ K.T @ V  # Multiply with the input and output matrices to find the translation
        token_out = decode_one_hot(one_hot_out, out)  # Decode the output one-hot vector to a token
        sentence_out += token_out + ' '  # Append the translated token to the output sentence
    return sentence_out.strip()  # Return the translated sentence

### Teste de tradução
A função de tradução aprimorada é testada com a frase "le chat est sous la table", verificando se ela é traduzida corretamente para "o gato está debaixo da mesa" usando as operações de matriz para uma tradução perfeita palavra por palavra.

In [27]:
translate("le chat est sous la table")

'the cat is under the table'

Esta abordagem aprimorada mostra como modelos de redes neurais podem traduzir idiomas representando o dicionário de tradução como matrizes e usando operações vetoriais.

**O próximo segmento de código apresenta conceitos que levam à implementação de "Atenção" em redes neurais:**

### Função Softmax para similaridade
É explicado que tokens similares terão vetores similares, e uma função softmax é adicionada à equação. Esta função é aplicada à saída da multiplicação da matriz do vetor de consulta `q` e da transposição da matriz `K`. A função softmax converte esses valores em probabilidades, enfatizando o token mais similar enquanto ainda considera os outros.

In [28]:
print('E_{table} = ', one_hot_in['table'])

E_{table} =  tensor([0., 0., 0., 0., 0., 1.])


$$
E_{table} =  \left[\begin{matrix}
  0 & 0 & 0 & 0 & 0 & 1\\\\\\\\\\\\
\end{matrix}\right]
\ \ \
$$

$$
E_{tables} =  \left[\begin{matrix}
  0 & 0 & 0 & 0 & 0 & 0.95\\\\
\end{matrix}\right]
$$


Our new equation is:
$$
softmax(q \cdot K^T) \cdot V
$$

Let's adjust using by the dimensionality of the query vector, and you'll get:

$$
softmax\left( \frac{q \cdot K^T}{\sqrt{d}} \right) \cdot V
$$


### Tradução com mecanismo de atenção
A função `translate` é modificada para usar a função softmax como uma forma de aplicar atenção. Primeiro, ela encontra o vetor one-hot para o token, depois aplica a função softmax ao produto escalar de `q` e `K.T`, dimensiona-o pela raiz quadrada da dimensionalidade (para fins de normalização) e, finalmente, multiplica isso por `V` para obter o vetor de saída.

In [31]:
def translate(sentence):
    """
    Translate a sentence using the attention mechanism represented by th K and V matrices
    The softmax function is used to calculate a weighted sum of the V vectors,
    focusing on the most relevant vector for translation
    """
    sentence_out = '' # Initialize the output sentence
    for token_in in tokenize(sentence): # Tokenize the input sentence
        q = one_hot_in[token_in] # Get the one-hot vector for the current token
        # Apply softmax to the scaled dot product of q and K.T, 
        # then multiply by V
        # This selects the most relevant translation vector from V
        out = torch.softmax(q @ K.T, dim=0) @ V
        token_out = decode_one_hot(one_hot_out, out) # Decode the output vector to a token
        sentence_out += token_out + ' ' # Append the translated token to the output sentence
    
    return sentence_out.strip()

In [32]:
translate("le chat est sous la table")

'the cat is under the table'

**Teste de tradução**: A função de tradução atualizada é testada para garantir que processe corretamente a frase de exemplo "le chat est sous la table", traduzindo-a para "the cat is under the table". Isso verifica se o mecanismo de atenção implementado usando softmax funciona conforme o esperado.

Esta etapa marca a progressão da tradução simples baseada em consulta para uma abordagem baseada em atenção, apresentando um componente-chave dos modelos modernos de tradução neural.

**A próxima parte do código demonstra uma melhoria no processo de tradução ao manipular todas as consultas em paralelo:**

### Criando a matriz 'Q'
A matriz `Q` é construída empilhando os vetores codificados one-hot de todos os tokens na sentença de entrada. Isso paraleliza o processo de preparação dos vetores de consulta, o que é mais eficiente do que fazê-lo sequencialmente.

In [33]:
# The sentence we wnat to translate
sentence = "le chat est sous la table"

In [35]:
# Stack all the one-hot encoded vectors for the tokens in the sentence to form the Q Matrix
Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])

In [36]:
print(Q)

tensor([[0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]])


$$
Q = \left[\begin{matrix}
  0 & 0 & 0 & 1 & 0 & 0 \\\\
  1 & 0 & 0 & 0 & 0 & 0 \\\\
  0 & 1 & 0 & 0 & 0 & 0 \\\\
  0 & 0 & 0 & 0 & 1 & 0 \\\\
  0 & 0 & 1 & 0 & 0 & 0 \\\\
  0 & 0 & 0 & 0 & 0 & 1 \\\\
\end{matrix}\right]
$$


$$
Attention(Q, K, V) = softmax\left( \frac{Q \cdot K^T}{\sqrt{d}} \right) V
$$


### Função translate atualizada
A função `translate` foi revisada para usar multiplicação de matrizes em toda a frase. Em vez de traduzir palavra por palavra, agora ela usa a matriz "Q" para executar a operação em paralelo para todas as palavras.

In [37]:
def translate(sentence):
    """
    Translate a sentence using matrix multiplication in parallel.
    This function replaces the iterative approach with a single matrix multiplication step,
    applying the attention mechanism across all tokens at once.
    """
    # Tokenize the sentence and stack the one-hot vectors to form the Q matrix
    Q = torch.stack([one_hot_in[token] for token in tokenize(sentence)])
    
    # Apply softmax to the dot product of Q and K.T and multiply by V
    # This will give us the output vectors for all tokens in parallel
    out = torch.softmax(Q @ K.T, 0) @ V
    
    # Decode each one-hot vector in the output to the corresponding token
    # And join the tokens to form the translated sentence
    return ' '.join([decode_one_hot(one_hot_out, o) for o in out])

In [38]:
# Test the function to ensure it produces the correct translation
translate("le chat est sous la table")

'the cat is under the table'

- **Melhoria de eficiência**: Ao aplicar operações à frase inteira de uma vez, essa abordagem simula um aspecto essencial do mecanismo de atenção real usado em redes neurais, que é processar vários componentes de dados de entrada em paralelo para uma computação mais rápida.

- **Saída do teste**: A função atualizada traduz corretamente a frase francesa "le chat est sous la table" para "o gato está debaixo da mesa", confirmando que a paralelização funciona efetivamente.

Essa otimização sugere as vantagens computacionais das operações de matriz em redes neurais, particularmente para tarefas como tradução, que se beneficiam do processamento paralelo.